In [47]:
import psycopg2
import json
from datetime import datetime as dt
import pandas as pd
import re 
from textblob import TextBlob 

In [16]:
p="datasucks"

In [17]:
conn = psycopg2.connect(user = "alexis",
                                  password = f"{p}",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")

In [18]:
cursor = conn.cursor()

In [39]:
tweet = []
followers = []
cursor.execute("select * from tweets")
tweets = cursor.fetchall()
if not tweets:
    print("empty")
for row in tweets:
    for col in row:
        if type(col) is dict:
            tweet.append(col['text'])
            followers.append(col['user']['followers_count'])

In [106]:
sentiment = int("0")
tweet_text = pd.DataFrame(tweet)
tweet_text.rename(columns = {0: "tweets"}, inplace = True)
# tweet_text["sentiment_test"] = sentiment
tweet_text.head()

,tweets
0,RT @BbyDmplng403: @LouiseLowrey There are so m...
1,RT @DanRodimer: Democrat frontrunner Michael B...
2,The latest COACH! https://t.co/Sz1NOFbZmK #ele...
3,THERE IS NO ONE TO VOTE FOR! \nCentrists deman...
4,RT @K_JeanPierre: Calling all voters!! Are you...


In [107]:
from itertools import islice

LIMIT = 50
for i,row in islice(tweet_text.iterrows(),LIMIT):
    tweet_text.tweets[i] = re.sub(r'@[A-Za-z0-9]+','',tweet_text.tweets[i])
    tweet_text.tweets[i] =  re.sub('https?://[A-Za-z0-9./]+','',tweet_text.tweets[i])
    tweet_text.tweets[i] =  re.sub('\n','',tweet_text.tweets[i])
    tweet_text.tweets[i] =  re.sub('RT :','',tweet_text.tweets[i])
    tweet_text.tweets[i] =  re.sub('RT:','',tweet_text.tweets[i])
    tweet_text.tweets[i] =  re.sub(r'_[A-Za-z0-9]+','',tweet_text.tweets[i])

In [108]:
tweet_text.head()

,tweets
0,There are so many hilarious comments in You ...
1,Democrat frontrunner Michael Bloomberg says h...
2,The latest COACH! #election2020 #misleadingtw...
3,THERE IS NO ONE TO VOTE FOR! Centrists demand ...
4,RT : Calling all voters!! Are you ready for No...


In [ ]:
sentiment_objects = [TextBlob(tweet) for tweet in tweets_no_urls]


In [35]:
# NRC
filepath = "NRC-Sentiment-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], skiprows=45, sep='\t')

In [14]:
emolex_df.emotion.unique()

array(['joy', 'negative', 'positive', 'sadness', 'surprise', 'trust',
       'anger', 'anticipation', 'disgust', 'fear'], dtype=object)

In [46]:
# def get_candidate(row):
#     candidates = []
#     text = row["text"].lower()
#     if "clinton" in text or "hillary" in text:
#         candidates.append("clinton")
#     if "trump" in text or "donald" in text:
#         candidates.append("trump")
#     if "sanders" in text or "bernie" in text:
#         candidates.append("sanders")
#     return ",".join(candidates)
# tweets["candidate"] = tweets.apply(get_candidate,axis=1)